In [1]:
# Importing Libraries

from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import warnings
warnings.filterwarnings('ignore')

In [2]:
url = "https://www.tripadvisor.com/Restaurant_Review-g293974-d3369977-Reviews-Nusret-Istanbul.html"
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
reg = requests.get(url, headers=headers)
if reg.status_code != 200 :
    print ('Hatalı/acılamayan sayfa ->', reg.status_code , url)
else :
    print ('Response Status :', reg.status_code)
soup = BeautifulSoup(reg.content, "lxml") 
print ('soup-', soup.prettify)

Response Status : 200
soup- <bound method Tag.prettify of <!DOCTYPE html>
<html lang="en" xmlns:og="http://opengraphprotocol.org/schema/"><head><meta content="text/html; charset=utf-8" http-equiv="content-type"/><link href="https://static.tacdn.com/favicon.ico?v2" id="favicon" rel="icon" type="image/x-icon"/><link color="#000000" href="https://static.tacdn.com/img2/brand_refresh/application_icons/mask-icon.svg" rel="mask-icon" sizes="any"/><meta content="#34e0a1" name="theme-color"/><meta content="telephone=no" name="format-detection"/><script type="text/javascript">window.taRollupsAreAsync = true;</script><link crossorigin="" href="https://static.tacdn.com/css2/webfonts/TripSans/TripSans.css?v1.002" rel="stylesheet"/><title>NUSRET, Istanbul - Besiktas - Menu, Prices &amp; Restaurant Reviews - Tripadvisor</title><meta content="TripAdvisor" property="al:ios:app_name"/><meta content="284876795" property="al:ios:app_store_id"/><meta content="284876795" name="twitter:app:id:ipad" property=

In [3]:
# toplam sayfa sayısı buluyoruz.
# pagenumber = soup.find("div", class_="prw_rup prw_common_responsive_pagination")
# for p in pagenumber.find_all("div", class_ = "pageNumbers"):
#    for a in p.find_all(class_ ="pageNum   cx_brand_refresh_phase2"):
#        link = a.find('a')
#        if link is not None and 'href' in link.attrs:
#                ara_link = a.get('href')
#        pages = a.text
#        print ('pages', pages)

In [4]:
def get_page_contents(url):
    page = requests.get(url, headers=headers)
    return BeautifulSoup(page.content, "lxml")

In [6]:
# her sayfada kullanıcılar tarafından yapılan yorumlara ait title, reviews, star bilgilerini topluyoruz

sayac = 0
baslık = []
konu = []
yıldız = []
for i in range(129):
    print ("i-", i)
    if i == 0 :
        url_1 = "https://www.tripadvisor.com/Restaurant_Review-g293974-d3369977-Reviews-Nusret-Istanbul.html"
    else :
        j =  i * 10
        url_1 = "https://www.tripadvisor.com/Restaurant_Review-g293974-d3369977-Reviews-or" + str(j) + "-Nusret-Istanbul.html"
    print ('url_1', url_1)
    page = requests.get(url_1, headers=headers)
    soup = BeautifulSoup(page.content, "lxml")
    
    ara_list = soup.find('div', { "class" : "ppr_rup ppr_priv_location_reviews_list_resp"})
    ara_list1 = ara_list.find( 'div', { "class" : "listContainer hide-more-mobile"})
    ara_list2 = ara_list1.find_all('div', { "class" : "prw_rup prw_reviews_review_resp"})
    
    for linkler in soup.find_all('div', class_="reviewSelector cx_brand_refresh_phase2"):
        title   = linkler.find('span', {"class":"noQuotes"})
        subject = linkler.find('p', {"class":"partial_entry"})
        # star
        rate = linkler.find('div', {'class': "ui_column is-9"})
        star = 0
        if rate.find('span', {'class':'ui_bubble_rating bubble_50'}):
            star = 5
        elif rate.find('span', {'class':'ui_bubble_rating bubble_40'}):
            star = 4
        elif rate.find('span', {'class':'ui_bubble_rating bubble_30'}):
            star = 3
        elif rate.find('span', {'class':'ui_bubble_rating bubble_20'}):
            star = 2
        elif rate.find('span', {'class':'ui_bubble_rating bubble_10'}):
            star = 1
        
        sayac += 1
        print ('title-', title.text)
        print ('konu-', subject.text)
        print ('yıldız-', star)
        print ('------------------', sayac , '--------------------')
        baslık.append(title.text)
        konu.append(subject.text)
        yıldız.append(star)
                                 
        nusret_reviews = pd.DataFrame({
            'title'   : baslık, 
            'reviews' : konu,
            'stars'   : yıldız
        })



i- 0
url_1 https://www.tripadvisor.com/Restaurant_Review-g293974-d3369977-Reviews-Nusret-Istanbul.html
title- what a disgrace
konu- The prices have doubled up while the quantity  and quality of the food deteriorated. DISAPPOINTED 😡😡
yıldız- 2
------------------ 1 --------------------
title- Way overrated!
konu- Despite the hype and the great ambience the food was quite overrated! The meat was too fatty and was not properly cooked to our order. The service was great and the team were super helpful and friendly.
yıldız- 3
------------------ 2 --------------------
title- Great Dining Experience
konu- I have visited Nusret inside the Grand Bazaar and also at the main location in Istanbul. Each time I have visited these restaurants I have never left disappointed. We have tried the lamb chops with fries, a steak dish, a hamburger, salad, etc. Everything has...always tasted fresh and the service is very good. When dining don't miss trying the baklava with vanilla ice cream. It's not to be mis

title- Don't bother
konu- We made our reservation one month in advance and arrived 10 min prior. This place looks like a burger king after they kick you out of the club. Staff doesn’t care if you have a reservation. There is nowhere to wait and people are giving...More
yıldız- 1
------------------ 21 --------------------
title- Amazing food
konu- Eating at nusret is a wonderful experience full of energy the and the atmosphere is full of young people
yıldız- 5
------------------ 22 --------------------
title- Best theatre and meat dishes in Turkey
konu- What a fabulous treat to enjoy such wonderful meat dishes in an amazing setting of the old grand bazaar in Istanbul...theatre at its best ....quality and tasty meat dishes ...friendly waiters ...great vibe and a wow factor ...worth every cent 👍
yıldız- 5
------------------ 23 --------------------
title- We couldn’t miss it!
konu- To visit Nusr-et steakhouse in Istanbul was our goal number one! So we just love it! It was tasty, the show w

title- Fine Dining
konu- Best tasting burgers and steak ever, this restaurant was better thank the one in Dubai more spacious and better priced food was amazing.
yıldız- 5
------------------ 51 --------------------
title- Best meat ever 
konu- Excellent food, tried a lot of things and all are excellent. Loved the breaded onions and baked potatoes as well. One of the best restaurants i ever had food in
yıldız- 5
------------------ 52 --------------------
title- Sami
konu- Not good at all, nothing special.
Tast below expectation. Over priced
Not recommended

Just belong to a famous face
yıldız- 2
------------------ 53 --------------------
title- Absolutely stunning
konu- Honestly famous for a reason. Food quality is great. Service was perfect. I had a “saltbay smoked” cocktail, and it was amazing.

I had the tomahawk medium rare, saltbay smoked, and their traditional paqlawa.. didn’t regret any of it.
yıldız- 5
------------------ 54 --------------------
title- Awesome
konu- Great meat, b

title- Better and much cheaper than their UAE branches 
konu- I live the UAE and Nusret is a very popular fine dining establishment. On Turkey it is a much more fun place and the prices are probably 25 percent of what you pay in the the UAE.

You'll need to book online a week or...More
yıldız- 5
------------------ 81 --------------------
title- Such a disappointment!!
konu- Made a reservation 2 weeks in advance and put in a request that it was for my birthday celebration and asked if possible to have a good table. When we arrived at the restaurant, I was confused for a second if it was the right...More
yıldız- 1
------------------ 82 --------------------
title- Lovely food...definitely worth a visit
konu- Came with family after hearing such great reviews. Restaurant was busy when we arrived but we were kindly seated upstairs within a couple of minutes of arriving. The waiter who served us recommended we order a selection from the menu so that we could taste...More
yıldız- 4
-----------

title- Superstar Nusret
konu- It became a pilgrimage site, the cue, waiting for a table, is unbelievable. It's chaotic, noisy and the stuff is not very friendly. The meet is excellent.....so nothing for a candle light or comfortable family dinner. Never have seen that before that people get that...More
yıldız- 2
------------------ 111 --------------------
title- Nusret Burger
konu- Had to wait for 15 minutes to get a table, which was not too bad. Tried the Nusret Burger first, and loved it! Beautifly seasoned, rested and served juicy! Out of curiosity, we had to try the Lokum Burger. It was an amazing SANDWICH (IMHO)...More
yıldız- 5
------------------ 112 --------------------
title- Mukkemel
konu- The best steak house in istanbul . A fine restaurant with an excellent choice of meat. Reservation should be made earlier. But still had to wait about 20 minutes
yıldız- 5
------------------ 113 --------------------
title- Birthday lunch 
konu- Veryh high profesional management 
We went for 

...

title- what a disgrace
konu- The prices have doubled up while the quantity  and quality of the food deteriorated. DISAPPOINTED 😡😡
yıldız- 2
------------------ 1196 --------------------
title- Way overrated!
konu- Despite the hype and the great ambience the food was quite overrated! The meat was too fatty and was not properly cooked to our order. The service was great and the team were super helpful and friendly.
yıldız- 3
------------------ 1197 --------------------
title- Great Dining Experience
konu- I have visited Nusret inside the Grand Bazaar and also at the main location in Istanbul. Each time I have visited these restaurants I have never left disappointed. We have tried the lamb chops with fries, a steak dish, a hamburger, salad, etc. Everything has...always tasted fresh and the service is very good. When dining don't miss trying the baklava with vanilla ice cream. It's not to be missed and tastes really good. I will keep returning when visiting Istanbul.More
yıldız- 5
---------

title- what a disgrace
konu- The prices have doubled up while the quantity  and quality of the food deteriorated. DISAPPOINTED 😡😡
yıldız- 2
------------------ 1216 --------------------
title- Way overrated!
konu- Despite the hype and the great ambience the food was quite overrated! The meat was too fatty and was not properly cooked to our order. The service was great and the team were super helpful and friendly.
yıldız- 3
------------------ 1217 --------------------
title- Great Dining Experience
konu- I have visited Nusret inside the Grand Bazaar and also at the main location in Istanbul. Each time I have visited these restaurants I have never left disappointed. We have tried the lamb chops with fries, a steak dish, a hamburger, salad, etc. Everything has...always tasted fresh and the service is very good. When dining don't miss trying the baklava with vanilla ice cream. It's not to be missed and tastes really good. I will keep returning when visiting Istanbul.More
yıldız- 5
---------

title- what a disgrace
konu- The prices have doubled up while the quantity  and quality of the food deteriorated. DISAPPOINTED 😡😡
yıldız- 2
------------------ 1236 --------------------
title- Way overrated!
konu- Despite the hype and the great ambience the food was quite overrated! The meat was too fatty and was not properly cooked to our order. The service was great and the team were super helpful and friendly.
yıldız- 3
------------------ 1237 --------------------
title- Great Dining Experience
konu- I have visited Nusret inside the Grand Bazaar and also at the main location in Istanbul. Each time I have visited these restaurants I have never left disappointed. We have tried the lamb chops with fries, a steak dish, a hamburger, salad, etc. Everything has...always tasted fresh and the service is very good. When dining don't miss trying the baklava with vanilla ice cream. It's not to be missed and tastes really good. I will keep returning when visiting Istanbul.More
yıldız- 5
---------

title- what a disgrace
konu- The prices have doubled up while the quantity  and quality of the food deteriorated. DISAPPOINTED 😡😡
yıldız- 2
------------------ 1256 --------------------
title- Way overrated!
konu- Despite the hype and the great ambience the food was quite overrated! The meat was too fatty and was not properly cooked to our order. The service was great and the team were super helpful and friendly.
yıldız- 3
------------------ 1257 --------------------
title- Great Dining Experience
konu- I have visited Nusret inside the Grand Bazaar and also at the main location in Istanbul. Each time I have visited these restaurants I have never left disappointed. We have tried the lamb chops with fries, a steak dish, a hamburger, salad, etc. Everything has...always tasted fresh and the service is very good. When dining don't miss trying the baklava with vanilla ice cream. It's not to be missed and tastes really good. I will keep returning when visiting Istanbul.More
yıldız- 5
---------

title- what a disgrace
konu- The prices have doubled up while the quantity  and quality of the food deteriorated. DISAPPOINTED 😡😡
yıldız- 2
------------------ 1276 --------------------
title- Way overrated!
konu- Despite the hype and the great ambience the food was quite overrated! The meat was too fatty and was not properly cooked to our order. The service was great and the team were super helpful and friendly.
yıldız- 3
------------------ 1277 --------------------
title- Great Dining Experience
konu- I have visited Nusret inside the Grand Bazaar and also at the main location in Istanbul. Each time I have visited these restaurants I have never left disappointed. We have tried the lamb chops with fries, a steak dish, a hamburger, salad, etc. Everything has...always tasted fresh and the service is very good. When dining don't miss trying the baklava with vanilla ice cream. It's not to be missed and tastes really good. I will keep returning when visiting Istanbul.More
yıldız- 5
---------

In [8]:
# nusret_reviews.to_csv('nusret_reviews_english.csv')

In [9]:
# df=pd.read_csv(r'C:\Users\Lenovo\Documents\GitHub\Proje_4\nusret_reviews_english.csv')

In [10]:
# df.head()

,Unnamed: 0,title,reviews,stars
0,0,what a disgrace,The prices have doubled up while the quantity ...,2
1,1,Way overrated!,Despite the hype and the great ambience the fo...,3
2,2,Great Dining Experience,I have visited Nusret inside the Grand Bazaar ...,5
3,3,MUST VISIT,We came here twice because on our first visit ...,5
4,4,dinner twice - once near the Grand Bazar outle...,Food at the Grand Bazar outlet was wonderful (...,4


In [11]:
# df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1285 entries, 0 to 1284
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1285 non-null   int64 
 1   title       1285 non-null   object
 2   reviews     1285 non-null   object
 3   stars       1285 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 40.3+ KB


## Mongo DB aktaralım

In [12]:
# nusret_reviews dataframe Mongo DB aktarıyoruz.

from pymongo import MongoClient

client = MongoClient()
db = client.tripAdvisor_NusrEt
collection = db.nusret_reviews

In [13]:
last_df = df.reset_index()
data_dict = last_df.to_dict("records")
collection.insert_many(data_dict)